In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
#visualisation libraries
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use(style="seaborn")

import datetime
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import folium
from folium import plugins
from sklearn.model_selection import TimeSeriesSplit
import warnings
warnings.simplefilter("ignore")
from prophet import Prophet
from statsmodels.graphics.tsaplots import plot_acf

In [10]:
df_India= pd.read_csv("df_India.csv" ,encoding='unicode_escape')
df_India

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,E4InternationalSupport,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality
0,India,2020/1/1,1380004385,35%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,India,2020/1/2,1380004385,35%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,India,2020/1/3,1380004385,35%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,India,2020/1/4,1380004385,35%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,India,2020/1/5,1380004385,35%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632,India,2021/9/24,1380004385,35%,1565696.0,559932709.0,1.124,401.844,1529080.0,1.097,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013284
633,India,2021/9/25,1380004385,35%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013280
634,India,2021/9/26,1380004385,35%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013278
635,India,2021/9/27,1380004385,35%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013276


In [8]:
#shift 21days mortality
df_India_mortality=df_India["mortality"]
df_India_mortality.drop([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21])

22     0.000000
23     0.000000
24     0.000000
25     0.000000
26     0.000000
         ...   
632    0.013284
633    0.013280
634    0.013278
635    0.013276
636    0.000000
Name: mortality, Length: 615, dtype: float64

In [14]:
#删除后21天
df_India.drop(df_India.index[615:636], inplace=True)
df_India

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,E4InternationalSupport,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality
0,India,2020/1/1,1380004385,35%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,India,2020/1/2,1380004385,35%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,India,2020/1/3,1380004385,35%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,India,2020/1/4,1380004385,35%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,India,2020/1/5,1380004385,35%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,India,2021/9/2,1380004385,35%,0.0,0.0,0.000,0.000,1675758.0,1.203,...,0.0,2.0,1.0,2.0,0.0,0.0,4.0,4.0,3.0,0.013369
611,India,2021/9/3,1380004385,35%,0.0,526535068.0,0.000,377.875,1654394.0,1.187,...,0.0,2.0,1.0,2.0,0.0,0.0,4.0,4.0,3.0,0.013362
612,India,2021/9/4,1380004385,35%,1704970.0,528240038.0,1.224,379.099,1621777.0,1.164,...,0.0,2.0,1.0,2.0,0.0,0.0,4.0,4.0,3.0,0.013354
613,India,2021/9/5,1380004385,35%,1818180.0,530058218.0,1.305,380.404,1630756.0,1.170,...,0.0,2.0,1.0,2.0,0.0,0.0,4.0,4.0,3.0,0.013345


In [15]:
df_India1=df_India.join(df_India_mortality,how="left",rsuffix="after21days")
df_India1

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality,mortalityafter21days
0,India,2020/1/1,1380004385,35%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,India,2020/1/2,1380004385,35%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,India,2020/1/3,1380004385,35%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,India,2020/1/4,1380004385,35%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
4,India,2020/1/5,1380004385,35%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,India,2021/9/2,1380004385,35%,0.0,0.0,0.000,0.000,1675758.0,1.203,...,2.0,1.0,2.0,0.0,0.0,4.0,4.0,3.0,0.013369,0.013369
611,India,2021/9/3,1380004385,35%,0.0,526535068.0,0.000,377.875,1654394.0,1.187,...,2.0,1.0,2.0,0.0,0.0,4.0,4.0,3.0,0.013362,0.013362
612,India,2021/9/4,1380004385,35%,1704970.0,528240038.0,1.224,379.099,1621777.0,1.164,...,2.0,1.0,2.0,0.0,0.0,4.0,4.0,3.0,0.013354,0.013354
613,India,2021/9/5,1380004385,35%,1818180.0,530058218.0,1.305,380.404,1630756.0,1.170,...,2.0,1.0,2.0,0.0,0.0,4.0,4.0,3.0,0.013345,0.013345


In [16]:
df_India1.to_csv("df_India_after21days",index=False)